# Image logging
Code to log image and configure camera to log images for a neural network training dataset

In [9]:
# boilerplate code
import carla 
import math 
import random 
import time 

# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

# Get the blueprint for the bycicle you want
vehicle_bp = bp_lib.find('vehicle.tesla.model3') 

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

## Create camera

In [6]:
import os
import cv2
import numpy as np
from queue import Queue
from datetime import datetime

# Create output directory if it doesn't exist
if not os.path.exists('output'):
   os.makedirs('output')

# Create camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1280')
camera_bp.set_attribute('image_size_y', '720')
camera_bp.set_attribute('fov', '110')

# Spawn camera relative to vehicle
camera_spawn = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_spawn, attach_to=vehicle)

image_queue = Queue()

def process_image(image):
   timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
   array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
   array = np.reshape(array, (image.height, image.width, 4))
   array = array[:, :, :3]
   
   # Save image
   cv2.imwrite(f'output/frame_{timestamp}.jpg', array)
   
   # Display image
   cv2.imshow('CARLA Camera', array)
   cv2.waitKey(1)

camera.listen(lambda image: image_queue.put(image))

# Start autopilot and save images for 10 seconds

In [7]:
# start the simulation
vehicle.set_autopilot(True)

start_time = time.time()
while time.time() - start_time < 10:
   image = image_queue.get()
   process_image(image)
   time.sleep(0.2)

camera.stop()
camera.destroy()
cv2.destroyAllWindows()

## Camera study

In [13]:

import cv2
import numpy as np
from queue import Queue

def setup_camera(resolution=(1280,720), pos=(1.5,0,2.4), fov=110):
    camera_bp = bp_lib.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(resolution[0]))
    camera_bp.set_attribute('image_size_y', str(resolution[1]))
    camera_bp.set_attribute('fov', str(fov))
    
    camera_spawn = carla.Transform(carla.Location(x=pos[0], y=pos[1], z=pos[2]))
    return world.spawn_actor(camera_bp, camera_spawn, attach_to=vehicle)

def study_image(camera, resolution, position, fov):
    image_queue = Queue()
    frames_shown = 0
    
    def process_frame(image):
        nonlocal frames_shown
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3].copy()
        
        info_text = [
            f"Resolution: {resolution[0]}x{resolution[1]}",
            f"Camera: x={position[0]:.1f}m, y={position[1]:.1f}m, z={position[2]:.1f}m",
            f"FOV: {fov} degrees"
        ]
        
        for i, text in enumerate(info_text):
            cv2.putText(array, text, (10, 30 + i*30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        
        cv2.imshow('CARLA Camera Study', array)
        key = cv2.waitKey(1)
        frames_shown += 1
        
        if key == 27:  # ESC key
            return False
        return True

    camera.listen(lambda image: image_queue.put(image))
    
    try:
        while frames_shown < 30:  # Show 30 frames
            if not image_queue.empty():
                image = image_queue.get()
                if not process_frame(image):
                    break
            time.sleep(0.1)
    finally:
        camera.stop()
        cv2.destroyAllWindows()

# Test cases
configs = [
    ((1280, 720), (1.5, 0, 2.4), 110),  # Default
    ((800, 600), (1.5, 0, 2.4), 110),   # Lower res
    ((1280, 720), (0, 0, 2.4), 110),    # Front
    ((1280, 720), (1.5, 1, 2.4), 110),  # Side
    ((1280, 720), (1.5, 0, 1.8), 110),  # Lower
    ((1280, 720), (1.5, 0, 2.4), 90),   # Narrow FOV
]

for resolution, position, fov in configs:
    print(f"\nTesting: {resolution}, {position}, FOV: {fov}")
    camera = setup_camera(resolution, position, fov)
    study_image(camera, resolution, position, fov)
    camera.destroy()
    time.sleep(2)


Testing: (1280, 720), (1.5, 0, 2.4), FOV: 110

Testing: (800, 600), (1.5, 0, 2.4), FOV: 110

Testing: (1280, 720), (0, 0, 2.4), FOV: 110

Testing: (1280, 720), (1.5, 1, 2.4), FOV: 110

Testing: (1280, 720), (1.5, 0, 1.8), FOV: 110

Testing: (1280, 720), (1.5, 0, 2.4), FOV: 90
